<h1>Affordability</h1>

In [ ]:
# Importing necessary libraries
import geopandas as gpd 
import folium
import pandas as pd
import sys

sys.path.insert(0, "../scripts/")
from helper_functions import convert_census_to_postcode

## Read in relevant data

In [ ]:
# Reading in preprocessed property data
property_df = pd.read_csv("../data/curated/categorised_distances.csv")
property_df = property_df[["Cost", "Bed", "Bath", "Parking", "Postcode"]]

# calculate median rent
median_rent_df = pd.DataFrame(property_df.groupby(["Postcode"])["Cost"].median()).reset_index()

## Dataset creation

### Calculating the average number of facilities per postcode

In [ ]:
# Creating dataframe of average facilities per postcode
avg_bed = property_df.groupby(by = "Postcode")["Bed"].mean()
avg_bath = property_df.groupby(by = "Postcode")["Bath"].mean()
avg_parking = property_df.groupby(by = "Postcode")["Parking"].mean()

facilities_df = pd.DataFrame()
facilities_df["Postcode"] = property_df["Postcode"].unique()
facilities_df["Average # Beds"] = avg_bed.to_list()
facilities_df["Average # Baths"] = avg_bath.to_list()
facilities_df["Average # Parking"] = avg_parking.to_list()
facilities_df.head()

In [ ]:
# Joining median rental price per postcode with average facilities
facilities_df = facilities_df.set_index("Postcode")
price_by_facility_df = median_rent_df.join(facilities_df, on = "Postcode")
price_by_facility_df = price_by_facility_df.set_index("Postcode")
price_by_facility_df.head()

### Obtaining census/income data

In [ ]:
# Read in census dataframes and convert to postcode mapping
census_df = pd.read_csv("../data/curated/census_data.csv")
sa2_postcode_map = pd.read_csv("../data/curated/sa2_postcode_mapping_2021.csv")
census_by_postcode_df = convert_census_to_postcode(census_df, sa2_postcode_map, "mean_no_zero")

In [ ]:
# Removing unnecessary columns from census data (only need columns containing data from 2021)
census_by_postcode_df = census_by_postcode_df[["postcode_2021", "tot_population_21", "avg_med_mortg_rep_21", "avg_med_person_inc_21", "avg_med_rent_21", 
                            "avg_med_hh_inc_21", "tot_avg_hh_size_21"]]

# Selecting columns required for assessing affordability
census_income_df = census_by_postcode_df[["postcode_2021", "avg_med_mortg_rep_21", "avg_med_hh_inc_21"]]

# Rename postcode column of census dataframe and setting to index
census_income_df = census_income_df.rename({"postcode_2021": "Postcode"}, axis = 1)
census_income_df = census_income_df.set_index("Postcode")

# Joining census income dataframe to dataframe containg price & facilities information
df = census_income_df.join(price_by_facility_df, on = "Postcode", how = "right")
df.head()

## Calculating affordability metric

In [ ]:
# If the number of facilities is 0 (i.e. for number of bedrooms or number of parking spaces) replace 0 value with arbitrary small value epsilon
EPSILON = 10**(-6)
 
df.loc[df["Average # Beds"] == 0, "Average # Beds"] = EPSILON
df.loc[df["Average # Parking"] == 0, "Average # Parking"] = EPSILON

In [ ]:
# Calculating weights for facilities (from previous correlation matrix know positive correlation exists between all 3 attributes and the cost)
bed_weight = df["Average # Beds"].corr(df["Cost"])
bath_weight = df["Average # Baths"].corr(df["Cost"])
parking_weight = df["Average # Parking"].corr(df["Cost"])

In [ ]:
# Taking the median rental price divided by the median income (multiply by 100 to get cost as a percentage of income)
df["(Cost / Income)%"] = (df["Cost"] / df["avg_med_hh_inc_21"]) * 100

# Taking the median mortage divided by the median income (multiply by 100 to get mortage as a percentage of income)
df["(Mortage / Income)%"] = (df["avg_med_mortg_rep_21"] / df["avg_med_hh_inc_21"]) * 100

# Calculating non-standardised affordability metric
df["Affordability"] = ((bed_weight * df["Average # Beds"]) + (bath_weight * df["Average # Baths"]) + (parking_weight * df["Average # Parking"]) + 
                        df["(Cost / Income)%"] + df["(Mortage / Income)%"])

# Standardised affordability metric
min_affordability = df.sort_values(by = "Affordability").head(1)["Affordability"].tolist()[0]
max_affordability = df.sort_values(by = "Affordability", ascending = False).head(1)["Affordability"].tolist()[0]
df["Standardised Affordability"] = (df["Affordability"] - min_affordability) / (max_affordability - min_affordability)

# Taking 1 - standardised affordability (to make sure metric ranks 0 as least affordable & 1 as most affordable)
df["Standardised Affordability"] = (1 - df["Standardised Affordability"])
df = df.reset_index()

# Create and export affordability dataframe
affordability_df = df[["Postcode", "Standardised Affordability"]]
affordability_df.to_csv("../data/curated/affordability.csv", index=False)

# Top 10 affordable postcodes
most_affordable_df = affordability_df.sort_values(by="Standardised Affordability", ascending=False).head(10)
most_affordable_df

## Graphing standardised affordability

In [ ]:
# Creating geoJSON file of postcode and geometry coordinates 
sf = gpd.read_file("../data/raw/external/POA_2021_AUST_GDA2020/POA_2021_AUST_GDA2020.shp")
postcodes = pd.read_csv("../data/raw/external/postcode.csv", names = ["POA_CODE21", "Name", "Area"])
postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)

# Converting the geometry shaape to to latitude and longitude
# TAKEN FROM TUTE 2 NOTEBOOK
sf["geometry"] = sf["geometry"].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf = sf[sf["POA_CODE21"].between("3000", "4000")]
sf["POA_CODE21"] = sf["POA_CODE21"].astype(int)

gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

**Affordability heatmap**

In [ ]:
# (y, x) since we want (lat, long)
gdf["centroid"] = gdf["geometry"].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[["Name", "POA_CODE21", "centroid"]].head()

affordability_df.rename({"Postcode": "POA_CODE21"})

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

c = folium.Choropleth(
            geo_data = geoJSON, # geoJSON 
            name = "choropleth", # name of plot
            data = affordability_df, # data source
            columns = ["Postcode", "Standardised Affordability"], # the columns required
            key_on = "properties.POA_CODE21", # this is from the geoJSON's properties
            fill_color = "YlOrRd", # color scheme
            nan_fill_color = "grey",
            legend_name = "Affordability"
        )

c.add_to(m)
m.save(f"../plots/affordability_heatmap.html")
m

**Most affordable heatmap**

In [ ]:
# Export most affordable heatmap
postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)
postcodes = postcodes.loc[postcodes["POA_CODE21"].isin(most_affordable_df["Postcode"].to_list())]

gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

m.add_child(folium.Choropleth(geo_data = geoJSON, name = "choropleth",))
m.save(f"../plots/most_affordable_heatmap.html")
m